In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.input_area pre {font-family: Consolas; font-size: 12pt; line-height: 140%;}</style>"))
display(HTML("<style>.output_area pre {font-family: Consolas; font-size: 12pt; line-height: 140%;}</style>"))

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
file_path = '../data/drinks.csv'
drinks = pd.read_csv(file_path)
drinks['continent'] = drinks['continent'].fillna('OT')

In [7]:
drinks.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,AS
1,Albania,89,132,54,4.9,EU
2,Algeria,25,0,14,0.7,AF
3,Andorra,245,138,312,12.4,EU
4,Angola,217,57,45,5.9,AF


1번 대륙별 평균 wine_servings를 탐색하라.

In [28]:
result = drinks.groupby('continent').mean()['wine_servings']
df = result.to_frame().reset_index()
df = df.rename(columns={'wine_servings': 'wine_servings_cont_avg'})
drinks = pd.merge(drinks, df, on='continent', how='outer')

In [29]:
drinks['wine_servings_cont_avg'] = drinks.groupby('continent')['wine_servings'].transform(np.mean)

In [30]:
drinks[['country', 'continent', 'wine_servings_cont_avg']].sample(5).head()

,country,continent,wine_servings_cont_avg
33,Sri Lanka,AS,9.068182
24,Nepal,AS,9.068182
34,Syria,AS,9.068182
22,Mongolia,AS,9.068182
65,Ireland,EU,142.222222


2번 국가별 모든 servings의 합을 계산한 total_servings라는 피처를 생성하라.

In [11]:
drinks['total_servings'] = drinks['beer_servings'] + drinks['spirit_servings'] + drinks['wine_servings']
drinks.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent,total_servings
0,Afghanistan,0,0,0,0.0,AS,0
1,Albania,89,132,54,4.9,EU,275
2,Algeria,25,0,14,0.7,AF,39
3,Andorra,245,138,312,12.4,EU,695
4,Angola,217,57,45,5.9,AF,319


3번 전체 평균보다 적은 알코올을 마시는 대륙 중, spirit를 가장 많이 마시는 국가를 찾아라.

In [13]:
total_mean = drinks.total_litres_of_pure_alcohol.mean()
continent_mean = drinks.groupby('continent')['total_litres_of_pure_alcohol'].mean()
continent_under_mean = continent_mean[continent_mean <= total_mean].index.tolist()
continent_under_mean
df_continent_under_mean = drinks.loc[drinks.continent.isin(continent_under_mean)]
most_spirit = df_continent_under_mean.loc[df_continent_under_mean['spirit_servings'].idxmax()]
most_spirit['country']

'Russian Federation'

4번 술 소비량 대비 알코올 비율을 구하라

In [15]:
drinks['alcohol_rate'] = drinks['total_litres_of_pure_alcohol'] / drinks['total_servings']
drinks['alcohol_rate'] = drinks['alcohol_rate'].fillna(0)

drinks.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent,total_servings,alcohol_rate
0,Afghanistan,0,0,0,0.0,AS,0,0.000000
1,Albania,89,132,54,4.9,EU,275,0.017818
2,Algeria,25,0,14,0.7,AF,39,0.017949
3,Andorra,245,138,312,12.4,EU,695,0.017842
4,Angola,217,57,45,5.9,AF,319,0.018495


5번 대륙별로 술 소비량 대비 알코올 비율을 계산하라.

In [17]:
continent_sum = drinks.groupby('continent').sum()
continent_sum['alcohol_rate_continent'] = continent_sum['total_litres_of_pure_alcohol'] / continent_sum['total_servings']
continent_sum = continent_sum[ 'alcohol_rate_continent']

continent_sum = continent_sum.reset_index()
continent_sum = continent_sum[['continent', 'alcohol_rate_continent']]

drinks = pd.merge(drinks, continent_sum, on='continent', how='outer')
drinks[['country', 'continent', 'alcohol_rate_continent']].head()

,country,continent,alcohol_rate_continent
0,Afghanistan,AS,0.020293
1,Bahrain,AS,0.020293
2,Bangladesh,AS,0.020293
3,Bhutan,AS,0.020293
4,Brunei,AS,0.020293
